In [1]:
import tensorflow as tf
import numpy as np
import tflearn
import random
import cv2
import os

from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tqdm import tqdm

In [2]:
TRAIN = "/home/ivan/PycharmProjects/ml/Cat_Dog/train/"
TEST = "/home/ivan/PycharmProjects/ml/Cat_Dog/train/"
SIZE = 50
LR = 0.001
MODEL_NAME = '{}.model'.format(LR)

In [3]:
def load_data(name):
    name = name.split('.')[0]
    if name == 'cat': 
        return [1,0]
    elif name == 'dog': 
        return [0,1]

In [4]:
def generate_train_data():
    train = []
    for image in tqdm(os.listdir(TRAIN)):
        label = load_data(image)
        full_path = os.path.join(TRAIN, image)
        image = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (SIZE, SIZE))
        train.append([np.array(image),np.array(label)])
    random.shuffle(train)
    return train

In [ ]:
def generate_test_data():
    test_data = []
    for image in tqdm(os.listdir(TEST)):
        full_path = os.path.join(TEST, image)
        image_class = image.split('.')[0]
        image = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (SIZE,SIZE))
        test_data.append([np.array(image), image_class])
    random.shuffle(test_data)
    return test_data

In [ ]:
train_data = generate_train_data() 

100%|██████████| 25000/25000 [00:28<00:00, 884.31it/s] 


In [ ]:
tf.reset_default_graph()

In [ ]:
cnn = input_data(shape=[None, SIZE, SIZE, 1], name='input')

cnn = conv_2d(cnn, 32, 5, activation='relu')
cnn = max_pool_2d(cnn, 5)

cnn = conv_2d(cnn, 64, 5, activation='relu')
cnn = max_pool_2d(cnn, 5)

cnn = conv_2d(cnn, 32, 5, activation='relu')
cnn = max_pool_2d(cnn, 5)

cnn = conv_2d(cnn, 64, 5, activation='relu')
cnn = max_pool_2d(cnn, 5)

cnn = conv_2d(cnn, 32, 5, activation='relu')
cnn = max_pool_2d(cnn, 5)

cnn = conv_2d(cnn, 32, 5, activation='relu')
cnn = max_pool_2d(cnn, 5)


cnn = fully_connected(cnn, 1024, activation='relu')
cnn = dropout(cnn, 0.8)

cnn = fully_connected(cnn, 2, activation='softmax')
cnn = regression(cnn, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(cnn, tensorboard_dir='log')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
train = train_data[:-500]
test = train_data[-500:]

In [ ]:
X = np.array([i[0] for i in train]).reshape(-1,SIZE,SIZE,1)
Y = np.array([i[1] for i in train])

test_x = np.array([i[0] for i in test]).reshape(-1,SIZE,SIZE,1)
test_y = np.array([i[1] for i in test])

In [ ]:
X.shape

(24500, 50, 50, 1)

In [ ]:
 model.fit({'input': X}, {'targets': Y}, n_epoch=7, 
          validation_set=({'input': test_x}, {'targets': test_y}), 
          snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 1710  | total loss: 0.46592 | time: 51.126s
| Adam | epoch: 005 | loss: 0.46592 - acc: 0.7814 -- iter: 11392/24500


In [ ]:
model.save(MODEL_NAME)

In [ ]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)


In [ ]:
model.predict(image)

In [ ]:
image = cv2.imread("/home/ivan/PycharmProjects/Cat_Dog_Vk/cat.jpg", 
                   cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (SIZE,SIZE))

In [ ]:
image = image.reshape(-1, SIZE, SIZE, 1)